In [1]:
using Combinatorics

cards = [1,2,3,4,5,6,7,8,9,10,11]

handPlayer = Array{Int8}(undef, 0)
handHouse = Array{Int}(undef,0)

takeCard() = rand(1:12,1)[1]

takeCard (generic function with 1 method)

In [10]:
MDP_states(hand) = [append!(copy(hand), i[1]) for i in collect(combinations(cards,1))]
#MDP_state(hand) = isa(hand, Array) ? hand : error("$hand is not an array")
MDP_legal_actions() = ["take", "pass"]
MDP_p(s,a,s_) =  a == "take" ? 1/13 : 0
MDP_terminal(hand) = sum(hand) == 21  || length(hand) == 4 ? true : false
MDP_r(s,a,s_) = sum(s_) == 21 || length(s_) == 4 ? 1 : -1

MDP_r (generic function with 1 method)

In [11]:
function iter_value(hand, r)
    V = Dict(s => 0 for s in MDP_states(hand))
    V_p = copy(V)
    while true
        for s in keys(V)
            V_p[s] = maximum([sum([MDP_p(s,a,s_)*(MDP_r(s,a,s_)+r*V[s_]) for s_ in MDP_states(hand)]) for a in MDP_legal_actions()])
            salir = true
            for s in keys(V)
                if V_p[s] > V[s]
                    salir = false
                    V[s] = V_p[s]
                end
            end
            if salir
                break
            end
        end
    end
    #Regresamos la politicaaaaa
    policy = Dict()
    for s in keys(V)
        temp = Dict(a => sum([MDP_p([1],a,s_)*V[s_] for s_ in MDP_states(handPlayer)]) for a in MDP_legal_actions())
        policy[s] = findmax(temp)[2]
        
    end
    return policy
end

iter_value (generic function with 1 method)

In [8]:
append!(handPlayer, takeCard())
handPlayer

2-element Array{Int8,1}:
 8
 6

In [12]:
iter_value(handPlayer,1)

InterruptException: InterruptException: